In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib
import time

In [2]:
#create a webdriver object and set options for headless browsing
options = Options()
options.headless = True
driver = webdriver.Chrome('./chromedriver',options=options)


In [3]:
def get_js_soup(url,driver):
    driver.get(url)

    time.sleep(5)

    redirectedUrl = driver.current_url

    if url != redirectedUrl:
        print("redirected URL : " + redirectedUrl)
        newUrl = redirectedUrl + "price-target/"
        print("new URL for Analysts rating : " + newUrl)
        driver.get(url)
    else:
        print("URL for Analysts rating: " + redirectedUrl)


    time.sleep(5)
    res_html = driver.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

In [4]:
''' More tidying
Sometimes the text extracted HTML webpage may contain javascript code and some style elements. 
This function removes script and style tags from HTML so that extracted text does not contain them.
'''
def remove_script(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    return soup

In [5]:
def get_mb_url(market,stocksymbol):
    base_url = "https://www.marketbeat.com/stocks/{market}/{stocksymbol}/price-target/"
    final_url = base_url.format(market = market, stocksymbol = stocksymbol)
    return final_url


In [6]:
url = get_mb_url("NASDAQ", "AAPL")
soup = get_js_soup(url,driver)

URL for Analysts rating: https://www.marketbeat.com/stocks/NASDAQ/AAPL/price-target/


In [41]:
soup = remove_script(soup)

In [49]:
soup = remove_script(soup)
table = soup.find("table", attrs={"class":"scroll-table sort-table fixed-left-column fixed-header"})
table_body = table.find('tbody')
rows = table_body.find_all('tr')

data = []

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele.split("➝")[-1].strip() for ele in cols if ele]) 
    #print(cols)

print (data[0])

['10/14/2020', 'Wedbush', 'Boost Price Target', 'Overweight', '$150.00', 'Low']
